# SIDA: Speaker Identification for Archives: Bill Clinton, notebook 1 of 2

## Before you begin, download the training corpus of audio files and the labeled data you'll need:

### Bill Clinton Audio: 
- http://xtra.arloproject.com/datasets/audio/Bill_Clinton_Speeches_UVA_Miller_Center.zip
- http://xtra.arloproject.com/datasets/audio/AAPB_Hand_Labeled/Bill_Clinton_wav.zip

### CSV that will allow you to create exercepts of Bill Clinton speaking from the above corpora: 
- https://github.com/hipstas/aapb-speaker-labels/blob/master/speaker_labels_randomized/Bill_Clinton/Bill_Clinton_5K_labels.csv

### Male and Female audio for a universal background model: 
- http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
- http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
- http://xtra.arloproject.com/datasets/aapb-ubm/Female_AAPB_171110.zip
- http://xtra.arloproject.com/datasets/aapb-ubm/Male_AAPB_171110.zip


In [1]:
## Import modules anos.chdir(class_dir_pathname)d create the training directory

import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/sida_classifier/

os.chdir('/sharedfolder/sida_classifier/')

In [2]:
## Downloading the CSV label file into the /sharedfolder/sida_classifier/ directory

!wget -N https://raw.githubusercontent.com/hipstas/aapb-speaker-labels/master/speaker_labels_randomized/Bill_Clinton/Bill_Clinton_5K_labels.csv

--2018-06-22 21:13:00--  https://raw.githubusercontent.com/hipstas/aapb-speaker-labels/master/speaker_labels_randomized/Bill_Clinton/Bill_Clinton_5K_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 426289 (416K) [text/plain]
Saving to: ‘Bill_Clinton_5K_labels.csv’

Bill_Clinton_5K_lab 100%[===================>] 416.30K  2.17MB/s    in 0.2s    

Last-modified header missing -- time-stamps turned off.
2018-06-22 21:13:00 (2.17 MB/s) - ‘Bill_Clinton_5K_labels.csv’ saved [426289/426289]



In [3]:
## Load 1-second labels

csv_path = '/sharedfolder/sida_classifier/Bill_Clinton_5K_labels.csv'

train_table_df = pd.read_csv(csv_path, names = ['Media file basename', 'Start', 'Duration', 'Label'])
os.chdir(class_dir_pathname)
train_table_df.head()

,Media file basename,Start,Duration,Label
0,CSPAN3_20160207_061500_Bill_Clinton_1992_New_H...,36.0,1.0,Bill Clinton
1,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
2,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
3,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
4,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton


In [4]:
## Choose variables to extract and assigning variables we'll use below

training_audio_dir_name = "Bill_Clinton"

labels_to_use = ['Bill Clinton']

media_dir_pathname = '/sharedfolder/sida_classifier/' + training_audio_dir_name

class_dir_pathname = '/sharedfolder/sida_classifier/_classes_' + training_audio_dir_name

try: os.mkdir(class_dir_pathname)
except: passfemales_5k.zip

In [6]:
## Downloading complete Bill Clinton audio collection

os.chdir(media_dir_pathname)

!wget -N http://xtra.arloproject.com/datasets/audio/Bill_Clinton_complete.zip

!unzip -o Bill_Clinton_complete.zip

--2018-06-22 21:14:08--  http://xtra.arloproject.com/datasets/audio/Bill_Clinton_complete.zip
Resolving xtra.arloproject.com (xtra.arloproject.com)... 146.6.168.12
Connecting to xtra.arloproject.com (xtra.arloproject.com)|146.6.168.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14459324548 (13G) [application/zip]
Saving to: ‘Bill_Clinton_complete.zip’

Bill_Clinton_comple 100%[===================>]  13.47G  19.3MB/s    in 12m 25s 

2018-06-22 21:26:33 (18.5 MB/s) - ‘Bill_Clinton_complete.zip’ saved [14459324548/14459324548]

Archive:  Bill_Clinton_complete.zip
   creating: Bill_Clinton/
  inflating: Bill_Clinton/youtube-43sUEFcD4Ro_880.mp4__46.0_1.0.wav  
  inflating: Bill_Clinton/youtube-43sUEFcD4Ro_880.mp4__32.0_1.0.wav  
  inflating: Bill_Clinton/youtube-43sUEFcD4Ro_880.mp4.wav  
  inflating: Bill_Clinton/youtube-43sUEFcD4Ro_720.mp4__36.0_1.0.wav  
  inflating: Bill_Clinton/youtube-43sUEFcD4Ro_720.mp4__24.0_1.0.wav  
  inflating: Bill_Clinton/youtube-43s

  inflating: Bill_Clinton/spe_1993_0205_clinton.mp3  
  inflating: Bill_Clinton/spe_1993_0129_clinton.mp3  
  inflating: Bill_Clinton/spe_1993_0120_clinton__525.0_1.0.wav  
  inflating: Bill_Clinton/spe_1993_0120_clinton__317.0_1.0.wav  
  inflating: Bill_Clinton/spe_1993_0120_clinton.mp3  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_880.mp4.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_720.mp4.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_560.mp4__40.0_1.0.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_560.mp4__32.0_1.0.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_560.mp4.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_400.mp4.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding_with_Bill_Clinton_2800.mp4.wav  
  inflating: Bill_Clinton/scacca-Get_Out_The_Vote_Redding

  inflating: Bill_Clinton/d445aaf4ad310964c5042e695ce9c35f.mp3  
  inflating: Bill_Clinton/d430f7d54d4892b6d372562caa1b34dd__401.0_1.0.wav  
  inflating: Bill_Clinton/d430f7d54d4892b6d372562caa1b34dd.mp3  
  inflating: Bill_Clinton/d3efb4f5f7bae8853196cda904e27e90__287.0_1.0.wav  
  inflating: Bill_Clinton/d3efb4f5f7bae8853196cda904e27e90__215.0_1.0.wav  
  inflating: Bill_Clinton/d3efb4f5f7bae8853196cda904e27e90.mp3  
  inflating: Bill_Clinton/d347635fbb54b6bc6c9bb9ed667e72f2.mp3  
  inflating: Bill_Clinton/d3056a47df951ec5a04cd831305ce8fc__25.0_1.0.wav  
  inflating: Bill_Clinton/d3056a47df951ec5a04cd831305ce8fc.mp3  
  inflating: Bill_Clinton/d130ec8eab76ebf8f2ba5caab8136460__19.0_1.0.wav  
  inflating: Bill_Clinton/d130ec8eab76ebf8f2ba5caab8136460.mp3  
  inflating: Bill_Clinton/d0c2e5adf2a3438839785a66cfe337d8__308.0_1.0.wav  
  inflating: Bill_Clinton/d0c2e5adf2a3438839785a66cfe337d8__304.0_1.0.wav  
  inflating: Bill_Clinton/d0c2e5adf2a3438839785a66cfe337d8__121.0_1.0.wav  
  in

  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__150.0_3.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__142.0_6.0__0.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__142.0_6.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__133.0_8.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__111.0_19.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__107.0_3.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-tx3513vq37__NHPR95203__102.0_4.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-s17sn01t72__NHPR95200__909.0_3.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-s17sn01t72__NHPR95200__766.32_135.0__99.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-s17sn01t72__NHPR95200__766.32_135.0__59.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-s17sn01t72__NHPR95200__766.32_135.0__32.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-s17sn01t72__NHPR95200__766.32_135.0.wav  


  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__416.0_98.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__414.0_100.0__93.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__414.0_100.0__38.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__414.0_100.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__398.0_15.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__347.0_51.0__27.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__347.0_51.0__11.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__347.0_51.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__346.0_51.0__32.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__346.0_51.0__25.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g18r__NHPR95189__346.0_51.0__12.0_1.0.wav  
  inflating: Bill_Clinton/cpb-aacip-503-6688g8g

...

  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_5_of_6_2008_400.mp4.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_4_of_6_2008_400.mp4__44.0_1.0.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_4_of_6_2008_400.mp4.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_3_of_6_2008_400.mp4__59.0_1.0.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_3_of_6_2008_400.mp4__41.0_1.0.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_3_of_6_2008_400.mp4.wav  
  inflating: Bill_Clinton/A_Conversation_with_Pres._William_J._Clinton_1_of_6_2008_400.mp4.wav  
  inflating: Bill_Clinton/AH-1290-1-Bill_Clinton_in_Eureka_1-16-08_880.mp4.wav  
  inflating: Bill_Clinton/AH-1290-1-Bill_Clinton_in_Eureka_1-16-08_720.mp4.wav  
  inflating: Bill_Clinton/AH-1290-1-Bill_Clinton_in_Eureka_1-16-08_560.mp4.wav  
  inflating: Bill_Clinton/AH-1290-1-Bill_Clinton

  inflating: Bill_Clinton/8e5fd5bf64a480785863576f9d9a7368.mp3  
  inflating: Bill_Clinton/8e34e36b4d05c4ff01c12762666fba01.mp3  
  inflating: Bill_Clinton/8bdbbf16991e7123f5ce57ee16ac8020__406.0_1.0.wav  
  inflating: Bill_Clinton/8bdbbf16991e7123f5ce57ee16ac8020.mp3  
  inflating: Bill_Clinton/8bda39751302226a0423a22a1c3c626f__63.0_1.0.wav  
  inflating: Bill_Clinton/8bda39751302226a0423a22a1c3c626f__56.0_1.0.wav  
  inflating: Bill_Clinton/8bda39751302226a0423a22a1c3c626f.mp3  
  inflating: Bill_Clinton/8b27b0d8310edbcb162d54cf75efdbdf__51.0_1.0.wav  
  inflating: Bill_Clinton/8b27b0d8310edbcb162d54cf75efdbdf__162.0_1.0.wav  
  inflating: Bill_Clinton/8b27b0d8310edbcb162d54cf75efdbdf.mp3  
  inflating: Bill_Clinton/8a9ef944f082d767f8f1c9c6e377dbd6__175.0_1.0.wav  
  inflating: Bill_Clinton/8a9ef944f082d767f8f1c9c6e377dbd6.mp3  
  inflating: Bill_Clinton/8a875532f0eae674e6db6b5c4fb84c62__270.0_1.0.wav  
  inflating: Bill_Clinton/8a875532f0eae674e6db6b5c4fb84c62.mp3  
  inflating: Bil

  inflating: Bill_Clinton/6fdf17ac9921a935edddf79532577079.mp3  
  inflating: Bill_Clinton/6d139a30b5df7ed68a3756c6834035de__145.0_1.0.wav  
  inflating: Bill_Clinton/6d139a30b5df7ed68a3756c6834035de.mp3  
  inflating: Bill_Clinton/6c3a47bf8e020d6c93a082d7e604a508__284.0_1.0.wav  
  inflating: Bill_Clinton/6c3a47bf8e020d6c93a082d7e604a508.mp3  
  inflating: Bill_Clinton/6c198f4673732c97dba0c27f7877359d.mp3  
  inflating: Bill_Clinton/6b80e8eace5274851ac69f7b90a9d82d__33.0_1.0.wav  
  inflating: Bill_Clinton/6b80e8eace5274851ac69f7b90a9d82d.mp3  
  inflating: Bill_Clinton/6b2c5411a5f09270cfd9fac28efede51__250.0_1.0.wav  
  inflating: Bill_Clinton/6b2c5411a5f09270cfd9fac28efede51__246.0_1.0.wav  
  inflating: Bill_Clinton/6b2c5411a5f09270cfd9fac28efede51__189.0_1.0.wav  
  inflating: Bill_Clinton/6b2c5411a5f09270cfd9fac28efede51.mp3  
  inflating: Bill_Clinton/693971879eba9fb69f742d8dafd4fa58__177.0_1.0.wav  
  inflating: Bill_Clinton/693971879eba9fb69f742d8dafd4fa58__139.0_1.0.wav  
  i

  inflating: Bill_Clinton/4474f205628fca1601c92e85564eff7b.mp3  
  inflating: Bill_Clinton/44338b0f6d25478a24a8d1cbb86fe3f0__253.0_1.0.wav  
  inflating: Bill_Clinton/44338b0f6d25478a24a8d1cbb86fe3f0__25.0_1.0.wav  
  inflating: Bill_Clinton/44338b0f6d25478a24a8d1cbb86fe3f0__219.0_1.0.wav  
  inflating: Bill_Clinton/44338b0f6d25478a24a8d1cbb86fe3f0__1.0_1.0.wav  
  inflating: Bill_Clinton/44338b0f6d25478a24a8d1cbb86fe3f0.mp3  
  inflating: Bill_Clinton/43fb2e54c181656ef0183f76fab83395__139.0_1.0.wav  
  inflating: Bill_Clinton/43fb2e54c181656ef0183f76fab83395.mp3  
  inflating: Bill_Clinton/435c89f1b894ea11c63faf927d38ac42__51.0_1.0.wav  
  inflating: Bill_Clinton/435c89f1b894ea11c63faf927d38ac42__182.0_1.0.wav  
  inflating: Bill_Clinton/435c89f1b894ea11c63faf927d38ac42.mp3  
  inflating: Bill_Clinton/42e2897102f29a7a8d03d9717569beeb__356.0_1.0.wav  
  inflating: Bill_Clinton/42e2897102f29a7a8d03d9717569beeb.mp3  
  inflating: Bill_Clinton/426b476a79a3bfa116f9bc371e4451e0__158.0_1.0.w

  inflating: Bill_Clinton/1bb5f67db9336b4a8c2571bf38802e3e.mp3  
  inflating: Bill_Clinton/1b4e7531e673ebe21a549cdc7b4a9cc7__99.0_1.0.wav  
  inflating: Bill_Clinton/1b4e7531e673ebe21a549cdc7b4a9cc7__210.0_1.0.wav  
  inflating: Bill_Clinton/1b4e7531e673ebe21a549cdc7b4a9cc7.mp3  
  inflating: Bill_Clinton/1b0baa6601cce702a3ec4ecb00ba036e.mp3  
  inflating: Bill_Clinton/1a00bb287f6cebe6e5f6a7db7c6ca0e7__95.0_1.0.wav  
  inflating: Bill_Clinton/1a00bb287f6cebe6e5f6a7db7c6ca0e7__2.0_1.0.wav  
  inflating: Bill_Clinton/1a00bb287f6cebe6e5f6a7db7c6ca0e7.mp3  
  inflating: Bill_Clinton/198814781bee1aa86f92023790906a0a__84.0_1.0.wav  
  inflating: Bill_Clinton/198814781bee1aa86f92023790906a0a.mp3  
  inflating: Bill_Clinton/18ec8a4a60f36cba9854c28a7e2e61ee.mp3  
  inflating: Bill_Clinton/184b1a19cf970d3449e4878fef5401da__206.0_1.0.wav  
  inflating: Bill_Clinton/184b1a19cf970d3449e4878fef5401da.mp3  
  inflating: Bill_Clinton/17f88acadc77588b2dd1f46a0b54ab38.mp3  
  inflating: Bill_Clinton/17b

In [8]:
os.chdir(class_dir_pathname)

!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Female_AAPB_171110.zip
!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Male_AAPB_171110.zip

!unzip -o females_5k.zip
!unzip -o males_5k.zip
!unzip -o Female_AAPB_171110.zip
!unzip -o Male_AAPB_171110.zip

--2018-06-22 21:37:31--  http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
Resolving xtra.arloproject.com (xtra.arloproject.com)... 146.6.168.12
Connecting to xtra.arloproject.com (xtra.arloproject.com)|146.6.168.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164730176 (157M) [application/zip]
Saving to: ‘females_5k.zip’

females_5k.zip      100%[===================>] 157.10M  19.5MB/s    in 8.5s    

2018-06-22 21:37:40 (18.5 MB/s) - ‘females_5k.zip’ saved [164730176/164730176]

--2018-06-22 21:37:40--  http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
Resolving xtra.arloproject.com (xtra.arloproject.com)... 146.6.168.12
Connecting to xtra.arloproject.com (xtra.arloproject.com)|146.6.168.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156500642 (149M) [application/zip]
Saving to: ‘males_5k.zip’

males_5k.zip        100%[===================>] 149.25M  20.0MB/s    in 7.5s  

  inflating: females_5k/sample-122507.mp3  
  inflating: females_5k/sample-038937.mp3  
  inflating: females_5k/sample-112310.mp3  
  inflating: females_5k/sample-050133.mp3  
  inflating: females_5k/sample-005452.mp3  
  inflating: females_5k/sample-174897.mp3  
  inflating: females_5k/sample-139944.mp3  
  inflating: females_5k/sample-041579.mp3  
  inflating: females_5k/sample-154670.mp3  
  inflating: females_5k/sample-164507.mp3  
  inflating: females_5k/sample-043891.mp3  
  inflating: females_5k/sample-167491.mp3  
  inflating: females_5k/sample-062710.mp3  
  inflating: females_5k/sample-021336.mp3  
  inflating: females_5k/sample-170897.mp3  
  inflating: females_5k/sample-067756.mp3  
  inflating: females_5k/sample-132326.mp3  
  inflating: females_5k/sample-095618.mp3  
  inflating: females_5k/sample-158640.mp3  
  inflating: females_5k/sample-016933.mp3  
  inflating: females_5k/sample-104538.mp3  
  inflating: females_5k/sample-080391.mp3  
  inflating: females_5k/sample-0

  inflating: females_5k/sample-175107.mp3  
  inflating: females_5k/sample-065269.mp3  
  inflating: females_5k/sample-018918.mp3  
  inflating: females_5k/sample-175399.mp3  
  inflating: females_5k/sample-024387.mp3  
  inflating: females_5k/sample-052100.mp3  
  inflating: females_5k/sample-021513.mp3  
  inflating: females_5k/sample-153209.mp3  
  inflating: females_5k/sample-083103.mp3  
  inflating: females_5k/sample-112978.mp3  
  inflating: females_5k/sample-016193.mp3  
  inflating: females_5k/sample-080036.mp3  
  inflating: females_5k/sample-160070.mp3  
  inflating: females_5k/sample-177946.mp3  
  inflating: females_5k/sample-138616.mp3  
  inflating: females_5k/sample-151528.mp3  
  inflating: females_5k/sample-099028.mp3  
  inflating: females_5k/sample-136704.mp3  
  inflating: females_5k/sample-043021.mp3  
  inflating: females_5k/sample-059970.mp3  
  inflating: females_5k/sample-061475.mp3  
  inflating: females_5k/sample-018671.mp3  
  inflating: females_5k/sample-0

  inflating: females_5k/sample-022844.mp3  
  inflating: females_5k/sample-116050.mp3  
  inflating: females_5k/sample-069496.mp3  
  inflating: females_5k/sample-005879.mp3  
  inflating: females_5k/sample-163717.mp3  
  inflating: females_5k/sample-042758.mp3  
  inflating: females_5k/sample-027158.mp3  
  inflating: females_5k/sample-114158.mp3  
  inflating: females_5k/sample-032208.mp3  
  inflating: females_5k/sample-022307.mp3  
  inflating: females_5k/sample-175309.mp3  
  inflating: females_5k/sample-010900.mp3  
  inflating: females_5k/sample-165320.mp3  
  inflating: females_5k/sample-105794.mp3  
  inflating: females_5k/sample-115866.mp3  
  inflating: females_5k/sample-122305.mp3  
  inflating: females_5k/sample-062347.mp3  
  inflating: females_5k/sample-071543.mp3  
  inflating: females_5k/sample-128716.mp3  
  inflating: females_5k/sample-050738.mp3  
  inflating: females_5k/sample-100445.mp3  
  inflating: females_5k/sample-193126.mp3  
  inflating: females_5k/sample-1

  inflating: females_5k/sample-073802.mp3  
  inflating: females_5k/sample-111198.mp3  
  inflating: females_5k/sample-135367.mp3  
  inflating: females_5k/sample-050111.mp3  
  inflating: females_5k/sample-153363.mp3  
  inflating: females_5k/sample-192120.mp3  
  inflating: females_5k/sample-170107.mp3  
  inflating: females_5k/sample-085364.mp3  
  inflating: females_5k/sample-112471.mp3  
  inflating: females_5k/sample-101027.mp3  
  inflating: females_5k/sample-146851.mp3  
  inflating: females_5k/sample-156972.mp3  
  inflating: females_5k/sample-106541.mp3  
  inflating: females_5k/sample-006047.mp3  
  inflating: females_5k/sample-072257.mp3  
  inflating: females_5k/sample-084595.mp3  
  inflating: females_5k/sample-095481.mp3  
  inflating: females_5k/sample-024776.mp3  
  inflating: females_5k/sample-149323.mp3  
  inflating: females_5k/sample-107129.mp3  
  inflating: females_5k/sample-030184.mp3  
  inflating: females_5k/sample-113914.mp3  
  inflating: females_5k/sample-0

...

  inflating: Male_AAPB_171110/cpb-aacip-500-kw57jd2k__403.0_413.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-kw57jd2k__403.0_413.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-kw57jf9g__1762.0_1772.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-m61bq97p__964.0_974.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-m61bq97p__964.0_974.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-mk658f44__1662.0_1672.0__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-mk658g2v__559.0_569.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-mk658g2v__559.0_569.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-ms3k203x__222.0_232.0__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-pz51m86g__1215.0_1225.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-pz51m86g__1215.0_1225.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-500-pz51m86g__1215.0_1225.0__4.0_5.0.wav  
  inflating: Male_

  inflating: Male_AAPB_171110/cpb-aacip-55-19s1sc16.h264__2145.0_2155.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-19s1sc16.h264__2145.0_2155.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-19s1scjj.h264__474.0_484.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-23vt51d1.h264__332.0_342.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-25x6bff8.h264__190.0_200.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-26m0d963.h264__276.0_286.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-26m0d963.h264__276.0_286.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-2n4zg6gb3c__cpb-aacip-55-34sj4k0t.mpeg2_.h264__195.0_205.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-2n4zg6gb3c__cpb-aacip-55-34sj4k0t.mpeg2_.h264__195.0_205.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-2v2c824n1t__cpb-aacip-55-63fxqggx.mpeg2_.h264__128.0_138.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-55-319s17s

  inflating: Male_AAPB_171110/cpb-aacip-60-65v6x4j4.h264__199.0_209.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-61-54kkwrvz.h264__62.0_72.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-61-74qjqckq.h264__1153.366_1163.366__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-61-752fr8h8.h264__285.0_295.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-61-752fr8h8.h264__285.0_295.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-61-913n663q.h264__462.0_472.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-62-9c6rx93n1h__netnola_honp_thefuture_doc_.h264__3298.0_3308.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-62-c53dz03c2w__netnola_dydn_10_doc_.h264__543.0_553.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-62-mp4vh5cw65__netnola_bljl_57_doc_.h264__129.0_139.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-62-mp4vh5cw65__netnola_bljl_57_doc_.h264__129.0_139.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-6

  inflating: Male_AAPB_171110/cpb-aacip-77-719kf35h.h264__1185.0_1195.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-72b8hxjv.h264__875.0_885.0__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-72b8hxjv.h264__875.0_885.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-72b8hxjv.h264__875.0_885.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-74cnq9rz.h264__291.0_301.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-7634vr19.h264__459.0_469.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-7634vstt.h264__1407.0_1417.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-7634vsvh.h264__1549.0_1559.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-773tzfpm.h264__1603.0_1613.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-77sn1861.h264__1269.0_1279.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-77sn1861.h264__310.0_320.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-77-784j24mk.h26

  inflating: Male_AAPB_171110/cpb-aacip-83-343r2hbz.h264__1696.0_1706.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-36547pqh.h264__2059.0_2069.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-36547pqh.h264__2059.0_2069.0__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-36547pqh.h264__2059.0_2069.0__2.0_3.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-36547pqh.h264__2059.0_2069.0__3.0_4.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-36547pqh.h264__2059.0_2069.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-43nvxfsp.h264__1654.0_1664.0__0.0_1.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-483jb338.h264__1214.0_1224.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-483jb338.h264__1591.0_1601.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-63stqz2w.h264__3257.0_3267.0__4.0_5.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-72p5j43x.h264__1732.0_1742.0__1.0_2.0.wav  
  inflating: Male_AAPB_171110/cpb-aacip-83-

In [11]:
%%capture
## Excerpting WAV clips corresponding to labels
# This cell will take some time.

os.chdir('/sharedfolder/sida_classifier/')

train_table_groups = train_table_df.groupby(['Media file basename', 'Label'])

for name, group in train_table_groups:
    list_of_lists = group.values.tolist()
    basename, start, duration, label = list_of_lists[0]
    filename = str(basename) + '.wav'
    media_path = os.path.join(media_dir_pathname, filename)
    if not os.path.isfile(media_path):
        filename = str(basename) + '.mp3'
        media_path = os.path.join(media_dir_pathname, filename)
    label_dir_pathname = str(os.path.join(class_dir_pathname, label.replace(' ','_')))
    if label in labels_to_use:
        subclip_pairs = []
        for row in list_of_lists:
            basename, start, duration, label = row
            subclip_pairs.append((float(start), float(duration)))
        try: subprocess.call(['mkdir', '-p', label_dir_pathname])
        except: pass
        try:
            attk.subclip_list(media_path, subclip_pairs, label_dir_pathname)
        except Exception as e: 
            print(e)

Before beginning this step, be sure all the male and female voices that you downloaded are unzipped into directories  in the _classes_Bill_Clinton directory like so: 

    /sharedfolder/sida_classifier/_classes_Bill_Clinton/Male_AAPB_171110/
    /sharedfolder/sida_classifier/_classes_Bill_Clinton/Female_AAPB_171110/

If you ran the cells above as they're written, you should be set.

In [12]:
%%capture
## Extract vowel segments from labeled audio clips
#### (This may take a while.)

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    try:
        attk.batch_extract_vowels(dir_name)
    except Exception as e:
        print("ERROR: " + dir_name)
        print(e)
    os.chdir(class_dir_pathname)

In [13]:
%%capture 
# The 'magic' command above suppresses this cell's output (including errors) 
# in order to avoid surpassing the browser's memory limit. Comment it out 
# to see output and errors.

## Extract features (MFCCs, deltas, and delta-deltas) from all clips, then write features to CSVs

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name, '_vowel_clips'))
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename, n_mfcc=30, n_fft=4096, freq_min=100, freq_max=16000)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

In [ ]:
## Continue to the next notebook to train and run the speaker ID classifier.